In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os

warnings.filterwarnings('ignore')

In [2]:
%%time
# Load from preprocessed data

years = ['2014', '2015', '2016', '2017', '2018']

df_raw = {}
for year in years:
    df_raw[year] = pd.read_feather('../data/Final_Divvy_data_'+year+'.feather')

CPU times: user 313 ms, sys: 1.06 s, total: 1.38 s
Wall time: 1.77 s


In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import widgets
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [4]:
# function to group total trips per day over stations
def group_over_dow_m(df):
    tmp_df = df[['month', 'dayofweek', 'year', 'total_in']]\
                .groupby(['month', 'dayofweek'])\
                .agg({'year': np.mean, 'total_in': np.sum}).reset_index()
    
    day_df = df[['month', 'dayofweek', 'day']]\
                .groupby(['month', 'dayofweek'])\
                .nunique().drop(['month', 'dayofweek'], axis=1).reset_index()
    
    tot_arr = np.array(tmp_df.total_in)
    days_arr = np.array(day_df.day)
    
    ave_per_day = tot_arr/days_arr
    return ave_per_day.reshape(12, 7).T

In [5]:
# list of months and dow
mx = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
dwy = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [6]:
def get_plot_info(df):
    tripcount = group_over_dow_m(df)
    hover_text = []
    for i in range(7):
        hy = dwy[i]
        tmp = []
        for j in range(12):
            hx = mx[j]
            tmp.append('Month: '+hx+'<br>Day of Week: '+hy+'<br>Total trips: '+str(int(tripcount[i][j])))
        hover_text.append(tmp)
    return tripcount, hover_text

In [7]:
# Prepare data
years = ['2014', '2015', '2016', '2017', '2018']

zmin = 0
zmax = 20000
cs = 'Portland'

traces = {}

for year in years:
    tp, ht = get_plot_info(df_raw[year])
    tmp_trace = go.Heatmap(
        name=year,
        z=tp,
        x=mx,
        y=dwy,
        text=ht,
        hoverinfo='text',
        colorscale=cs,
#         reversescale=True,
        zmin=zmin,
        zmax=zmax,
        zauto=False,
        zsmooth='best'
    )
    traces[year] = tmp_trace

In [73]:
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['title'] = dict(
        text='Bike demand of all stations',
        font=dict(
            family='Droid Serif, serif',
            size=30,
            color='Black'
        ),
    )
figure['layout']['xaxis'] = dict(
        title='Month',
        titlefont=dict(
            family='Arial, sans-serif',
            size=25,
            color='grey'
        ),
        showticklabels=True,
        tickangle=-45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
figure['layout']['yaxis'] = dict(
        title='Day of Week',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='grey'
        ),
        showticklabels=True,
        tickangle=-45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
figure['layout']['margin'] = go.layout.Margin(
    l=60,
    r=10,
    b=10,
    t=50,
    pad=6
)
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '2014',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True},
                         'fromcurrent': True, 'transition': {'duration': 1000, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [74]:
# Prepare data
figure['data'] = [traces['2014']]

In [75]:
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    frame['data'].append(traces[year])

    figure['frames'].append(frame)

    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [76]:
SHOW = False
if SHOW:
    iplot(figure, config={'displayModeBar': False})
else:
    plot(figure, config={'displayModeBar': False}, filename='trip_count_heatmap.html')